In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import os
os.chdir('data/coherent-11-07-2022/csv/')

In [ ]:
# List all file in folder
csv_list = [x for x in os.listdir('.') if x.endswith('.csv')]
csv_list

In [7]:
# Select or comment out files to be read in
df_csv = [# 'allergies.csv',
        # 'careplans.csv',
        'conditions.csv',
        'devices.csv',
        'encounters.csv',
        # 'imaging_studies.csv',
        #'immunizations.csv',
        'medications.csv',
        'observations.csv',
        # 'organizations.csv',
        'patients.csv',
        # 'payers.csv',
        # 'payer_transitions.csv',
        'procedures.csv',
        'providers.csv',
        # 'supplies.csv'
        ]


In [8]:
# Create a dictionary of dfs for each selected csv
d = {}
for x in df_csv:
    d[x.split('.')[0]] = pd.read_csv(x)

# Lengths of the respective dataframes to join
df_lengths = pd.DataFrame({'df': d.keys(), 'df_length': [d[x].shape[0] for x in d.keys()]})
df_lengths

,df,df_length
0,conditions,35874
1,devices,552
2,encounters,285339
3,medications,371210
4,observations,1480409
5,patients,3539
6,procedures,134385
7,providers,13920


In [21]:
# Rename columns for join
d['patients'].rename(columns={'Id': 'PATIENT'}, inplace=True)
d['patients'].columns

Index(['PATIENT', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT',
       'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE',
       'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE',
       'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES',
       'HEALTHCARE_COVERAGE'],
      dtype='object')

In [22]:
# Rename columns for join
d['encounters'].rename(columns={'Id': 'ENCOUNTER', 'START': 'Encounter_start', 'STOP': 'Encounter_stop',
                                'REASONDESCRIPTION': 'Encounter_reason', 'REASONCODE': 
                                'Encounter_reasoncode', 'CODE': 'Encounter_classcode', 'DESCRIPTION': 
                                'Encounter_type'}, inplace=True)
d['encounters'].columns

Index(['ENCOUNTER', 'Encounter_start', 'Encounter_stop', 'PATIENT',
       'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS',
       'Encounter_classcode', 'Encounter_type', 'BASE_ENCOUNTER_COST',
       'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'Encounter_reasoncode',
       'Encounter_reason'],
      dtype='object')

In [23]:
# Rename columns for join
d['conditions'].rename(columns={'START': 'CONDITION_START', 'STOP': 'CONDITION_STOP',
                                'CODE': 'CONDITION_CODE', 'DESCRIPTION': 'CONDITION_DESCRIPTION'}, 
                                inplace=True)
conditions = pd.DataFrame(d['conditions'])
conditions.drop(['PATIENT'], axis = 1, inplace=True)

In [24]:
# Merge 'patients' and 'encounter' on patient column
large_df = pd.DataFrame(d['patients'].merge(d['encounters'], how='outer', on='PATIENT'))
large_df.columns

Index(['PATIENT', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT',
       'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE',
       'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE',
       'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES',
       'HEALTHCARE_COVERAGE', 'ENCOUNTER', 'Encounter_start', 'Encounter_stop',
       'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS',
       'Encounter_classcode', 'Encounter_type', 'BASE_ENCOUNTER_COST',
       'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'Encounter_reasoncode',
       'Encounter_reason'],
      dtype='object')

In [25]:
# Drop irrelevant columns
large_df.drop(['SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 
                'SUFFIX', 'MAIDEN', 'ADDRESS', 'COUNTY', 'ZIP', 'LAT', 'LON', 
                'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'BASE_ENCOUNTER_COST',
                'TOTAL_CLAIM_COST', 'PAYER_COVERAGE'], axis = 1, inplace = True)
large_df.columns

Index(['PATIENT', 'BIRTHDATE', 'DEATHDATE', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'CITY', 'STATE', 'ENCOUNTER', 'Encounter_start',
       'Encounter_stop', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS',
       'Encounter_classcode', 'Encounter_type', 'Encounter_reasoncode',
       'Encounter_reason'],
      dtype='object')

In [26]:
# Merge current df with 'conditions' on the encounter column
large_df = large_df.merge(conditions, how = 'outer', on = 'ENCOUNTER')

In [27]:
# Rename columns for join, drop redundant columns 
d['observations'].rename(columns={'DATE': 'OBSERVATION_DATE', 'CODE': 'OBSERVATION_CODE', 
                                  'DESCRIPTION': 'OBSERVATION_DESCRIPTION'}, inplace=True)

observations = pd.DataFrame(d['observations'])

observations.drop(['PATIENT'], axis = 1, inplace=True)

In [28]:
# Merge current df with 'observations' on the encounter column
large_df = large_df.merge(observations, how = 'outer', on = 'ENCOUNTER')
large_df.columns

Index(['PATIENT', 'BIRTHDATE', 'DEATHDATE', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'CITY', 'STATE', 'ENCOUNTER', 'Encounter_start',
       'Encounter_stop', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS',
       'Encounter_classcode', 'Encounter_type', 'Encounter_reasoncode',
       'Encounter_reason', 'CONDITION_START', 'CONDITION_STOP',
       'CONDITION_CODE', 'CONDITION_DESCRIPTION', 'OBSERVATION_DATE',
       'OBSERVATION_CODE', 'OBSERVATION_DESCRIPTION', 'VALUE', 'UNITS',
       'TYPE'],
      dtype='object')

In [46]:
# Rename columns for join, drop redundant columns
d['medications'].rename(columns={'START': 'MED_START', 'STOP': 'MED_STOP', 'CODE': 'MED_CODE', 
                                 'DESCRIPTION': 'MEDICATION', 'REASONDESCRIPTION': 'MEDREASON'}, inplace=True)

medications = pd.DataFrame(d['medications'])

medications.drop(['PATIENT', 'PAYER', 'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 
                  'TOTALCOST', 'REASONCODE'], axis = 1, inplace=True)

In [47]:
medications['MEDREASON'].str.contains('carcinoma | neoplasm | cancer | malignant').sum()

12801

In [48]:
# Merge current df with 'medicaitons' on the encounter column
large_df = large_df.merge(medications, how = 'outer', on = 'ENCOUNTER')
large_df.columns

Index(['PATIENT', 'BIRTHDATE', 'DEATHDATE', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'CITY', 'STATE', 'ENCOUNTER', 'Encounter_start',
       'Encounter_stop', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS',
       'Encounter_classcode', 'Encounter_type', 'Encounter_reasoncode',
       'Encounter_reason', 'CONDITION_START', 'CONDITION_STOP',
       'CONDITION_CODE', 'CONDITION_DESCRIPTION', 'OBSERVATION_DATE',
       'OBSERVATION_CODE', 'OBSERVATION_DESCRIPTION', 'VALUE', 'UNITS', 'TYPE',
       'MED_START', 'MED_STOP', 'MED_CODE', 'MEDICATION', 'MEDREASON'],
      dtype='object')

In [51]:
# List of date-based columns to convert to datetime
date_columns = ['BIRTHDATE', 'DEATHDATE', 'OBSERVATION_DATE', 'MED_START', 'MED_STOP', 
                'Encounter_start', 'Encounter_stop', 'CONDITION_START']

# Convert dates to datetime format
for col in date_columns:
    large_df[col] = pd.to_datetime(large_df[col])
    large_df[col] = large_df[col].dt.date.astype('datetime64[ns]')   